In [1]:
import numpy as np
import pandas as pd

# Load X (features)
X = pd.read_csv("data.csv", header=None).to_numpy(dtype=float)

# Load y (labels)
y = pd.read_csv("label.csv", header=None).iloc[:, 0].to_numpy()

# K = number of unique labels
K = len(np.unique(y))

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Number of classes (K):", K)
print("Unique labels:", np.unique(y))

X shape: (10000, 784)
y shape: (10000,)
Number of classes (K): 10
Unique labels: [0 1 2 3 4 5 6 7 8 9]


In [2]:
import numpy as np

def euclid_dist(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.linalg.norm(a - b))

def cos_dist(a: np.ndarray, b: np.ndarray) -> float:
    na, nb = np.linalg.norm(a), np.linalg.norm(b)
    if na == 0.0 or nb == 0.0:
        return 1.0
    return 1.0 - float(np.dot(a, b) / (na * nb))

def jaccard_dist(a: np.ndarray, b: np.ndarray) -> float:
    mn = np.minimum(a, b).sum()
    mx = np.maximum(a, b).sum()
    if mx == 0.0:
        return 0.0
    return 1.0 - float(mn / mx)

def min_shift_nonnegative(X: np.ndarray) -> np.ndarray:
    mins = X.min(axis=0)
    shift = np.where(mins < 0, -mins, 0.0)
    return X + shift



In [3]:
import time

def run_kmeans(X: np.ndarray,
               k: int,
               dist_fn,
               max_iter: int = 500,
               seed: int = 7,
               atol: float = 1e-8):
    rng = np.random.default_rng(seed)
    n = len(X)

    # init centroids by sampling k rows
    init_idx = rng.choice(n, size=k, replace=False)
    C = X[init_idx].astype(float)

    def assign_labels(X, C, dist_fn):
        dmat = np.stack([[dist_fn(x, c) for c in C] for x in X], axis=0)
        return dmat.argmin(axis=1)

    def compute_sse(X, labels, C, dist_fn):
        return float(np.sum([
            dist_fn(X[i], C[labels[i]])**2 for i in range(len(X))
        ]))

    prev_sse = np.inf
    sse_trace = []
    stop_reason = "max_iter"
    t0 = time.perf_counter()

    for _ in range(1, max_iter + 1):
        labels = assign_labels(X, C, dist_fn)

        # update centroids (keep old if cluster empty)
        newC = C.copy()
        for j in range(k):
            pts = X[labels == j]
            if len(pts) > 0:
                newC[j] = pts.mean(axis=0)

        # stop rule 1: no centroid movement
        if np.allclose(newC, C, atol=atol):
            C = newC
            cur_sse = compute_sse(X, labels, C, dist_fn)
            sse_trace.append(cur_sse)
            stop_reason = "no_centroid_change"
            break

        C = newC
        cur_sse = compute_sse(X, labels, C, dist_fn)
        sse_trace.append(cur_sse)

        # stop rule 2: SSE increased
        if cur_sse > prev_sse - 1e-12:
            stop_reason = "sse_increase"
            break

        prev_sse = cur_sse

    elapsed = time.perf_counter() - t0
    return {
        "centroids": C,
        "labels": labels,
        "sse": sse_trace[-1],
        "it": len(sse_trace),
        "time": elapsed,
        "sse_trace": sse_trace,
        "stop_reason": stop_reason
    }


In [4]:
def majority_vote_accuracy(labels: np.ndarray, y: np.ndarray) -> float:
    k = labels.max() + 1
    preds = np.empty_like(labels)
    for j in range(k):
        mask = (labels == j)
        if not np.any(mask):
            continue
        cls, cnts = np.unique(y[mask], return_counts=True)
        preds[mask] = cls[cnts.argmax()]
    return float((preds == y).mean())


In [5]:
res_euclid = run_kmeans(
    X=X,
    k=K,
    dist_fn=euclid_dist,
    max_iter=500,
    seed=7
)

acc_euclid = majority_vote_accuracy(res_euclid["labels"], y)

print("=== EUCLIDEAN K-MEANS RESULTS ===")
print("Final SSE:", res_euclid["sse"])
print("Iterations:", res_euclid["it"])
print("Time (sec):", res_euclid["time"])
print("Stop reason:", res_euclid["stop_reason"])
print("Accuracy (majority vote):", acc_euclid)


=== EUCLIDEAN K-MEANS RESULTS ===
Final SSE: 25398937131.455795
Iterations: 59
Time (sec): 62.31521449999855
Stop reason: no_centroid_change
Accuracy (majority vote): 0.5878


In [6]:
n = len(X)
print("SSE:", res_euclid["sse"])
print("SSE per point:", res_euclid["sse"]/n)

# Average Euclidean distance to assigned centroid (not squared)
def avg_dist(res):
    C = res["centroids"]; labels = res["labels"]
    s = 0.0
    for i, x in enumerate(X):
        s += euclid_dist(x, C[labels[i]])
    return s / n

print("Avg (unsquared) distance per point:", avg_dist(res_euclid))

# See convergence trend
print("SSE first -> last:", res_euclid["sse_trace"][0], "->", res_euclid["sse_trace"][-1], 
      " (iters:", res_euclid["it"], ", stop:", res_euclid["stop_reason"], ")")


SSE: 25398937131.455795
SSE per point: 2539893.7131455797
Avg (unsquared) distance per point: 1565.4211841121464
SSE first -> last: 28249280587.548405 -> 25398937131.455795  (iters: 59 , stop: no_centroid_change )


In [7]:
res_cosine = run_kmeans(
    X=X,
    k=K,
    dist_fn=cos_dist,
    max_iter=500,
    seed=7
)

acc_cosine = majority_vote_accuracy(res_cosine["labels"], y)

print("=== COSINE K-MEANS RESULTS ===")
print("Final SSE:", res_cosine["sse"])
print("Iterations:", res_cosine["it"])
print("Time (sec):", res_cosine["time"])
print("Stop reason:", res_cosine["stop_reason"])
print("Accuracy (majority vote):", acc_cosine)


=== COSINE K-MEANS RESULTS ===
Final SSE: 688.2480129528869
Iterations: 21
Time (sec): 49.470101399994746
Stop reason: sse_increase
Accuracy (majority vote): 0.6516


In [8]:
res_jaccard = run_kmeans(
    X=X,
    k=K,
    dist_fn=jaccard_dist,
    max_iter=500,
    seed=7
)

acc_jaccard = majority_vote_accuracy(res_jaccard["labels"], y)

print("=== JACCARD K-MEANS RESULTS ===")
print("Final SSE:", res_jaccard["sse"])
print("Iterations:", res_jaccard["it"])
print("Time (sec):", res_jaccard["time"])
print("Stop reason:", res_jaccard["stop_reason"])
print("Accuracy (majority vote):", acc_jaccard)


=== JACCARD K-MEANS RESULTS ===
Final SSE: 3670.909111553253
Iterations: 21
Time (sec): 43.797116599998844
Stop reason: sse_increase
Accuracy (majority vote): 0.6401


In [9]:
import time
import numpy as np

def run_kmeans_forced(X, k, dist_fn, rule, max_iter=500, seed=7, atol=1e-8):
    """
    rule: 'no_change' | 'sse_increase' | 'max_iter'
    Forces ONLY the specified stopping rule to be active.
    """
    rng = np.random.default_rng(seed)
    n = len(X)
    C = X[rng.choice(n, size=k, replace=False)].astype(float)

    def assign_labels(X, C, dist_fn):
        dmat = np.stack([[dist_fn(x, c) for c in C] for x in X], axis=0)
        return dmat.argmin(axis=1)

    def compute_sse(X, labels, C, dist_fn):
        return float(np.sum([(dist_fn(X[i], C[labels[i]])**2) for i in range(len(X))]))

    prev_sse = np.inf
    sse_trace = []
    t0 = time.perf_counter()
    stop_reason = None

    for it in range(1, max_iter+1):
        labels = assign_labels(X, C, dist_fn)

        newC = C.copy()
        for j in range(k):
            pts = X[labels == j]
            if len(pts) > 0:
                newC[j] = pts.mean(axis=0)

        cur_sse = compute_sse(X, labels, newC, dist_fn)
        sse_trace.append(cur_sse)

        if rule == 'no_change':
            if np.allclose(newC, C, atol=atol):
                stop_reason = 'no_centroid_change'
                C = newC
                break
        elif rule == 'sse_increase':
            if cur_sse > prev_sse - 1e-12:
                stop_reason = 'sse_increase'
                C = newC
                break
        elif rule == 'max_iter':
            # just keep looping; we'll stop after reaching max_iter
            pass

        C = newC
        prev_sse = cur_sse

    if rule == 'max_iter' and stop_reason is None:
        stop_reason = 'max_iter_reached'

    elapsed = time.perf_counter() - t0
    return {
        "centroids": C,
        "labels": labels,
        "sse": sse_trace[-1],
        "it": len(sse_trace),
        "time": elapsed,
        "stop_reason": stop_reason
    }


In [10]:
eu_nochange = run_kmeans_forced(X, K, euclid_dist, rule='no_change', max_iter=500, seed=7)
print("EUCLIDEAN (no_change) -> SSE:", eu_nochange["sse"], 
      "iters:", eu_nochange["it"], "time:", eu_nochange["time"], 
      "stop:", eu_nochange["stop_reason"])


EUCLIDEAN (no_change) -> SSE: 25398937131.455795 iters: 59 time: 71.2903707000005 stop: no_centroid_change


In [11]:
eu_sseinc = run_kmeans_forced(X, K, euclid_dist, rule='sse_increase', max_iter=500, seed=7)
print("EUCLIDEAN (sse_increase) -> SSE:", eu_sseinc["sse"], 
      "iters:", eu_sseinc["it"], "time:", eu_sseinc["time"], 
      "stop:", eu_sseinc["stop_reason"])


EUCLIDEAN (sse_increase) -> SSE: 25398937131.455795 iters: 500 time: 677.3631806000049 stop: None


In [13]:
eu_max100 = run_kmeans_forced(X, K, euclid_dist, rule='max_iter', max_iter=100, seed=7)
print("EUCLIDEAN (max_iter=100) -> SSE:", eu_max100["sse"], 
      "iters:", eu_max100["it"], "time:", eu_max100["time"], 
      "stop:", eu_max100["stop_reason"])


EUCLIDEAN (max_iter=100) -> SSE: 25398937131.455795 iters: 100 time: 124.41878369999904 stop: max_iter_reached


In [14]:
cos_nochange = run_kmeans_forced(
    X, K, dist_fn=cos_dist,
    rule='no_change', max_iter=500, seed=7
)

print("COSINE (no_change) -> SSE:", cos_nochange["sse"],
      "iters:", cos_nochange["it"], "time:", cos_nochange["time"],
      "stop:", cos_nochange["stop_reason"])


COSINE (no_change) -> SSE: 688.2960352545117 iters: 39 time: 95.65580069999851 stop: no_centroid_change


In [15]:
cos_sseinc = run_kmeans_forced(
    X, K, dist_fn=cos_dist,
    rule='sse_increase', max_iter=500, seed=7
)

print("COSINE (sse_increase) -> SSE:", cos_sseinc["sse"],
      "iters:", cos_sseinc["it"], "time:", cos_sseinc["time"],
      "stop:", cos_sseinc["stop_reason"])


COSINE (sse_increase) -> SSE: 688.2480129528869 iters: 21 time: 50.52831010000227 stop: sse_increase


In [16]:
cos_max100 = run_kmeans_forced(
    X, K, dist_fn=cos_dist,
    rule='max_iter', max_iter=100, seed=7
)

print("COSINE (max_iter=100) -> SSE:", cos_max100["sse"],
      "iters:", cos_max100["it"], "time:", cos_max100["time"],
      "stop:", cos_max100["stop_reason"])


COSINE (max_iter=100) -> SSE: 688.2960352545117 iters: 100 time: 240.0778094999987 stop: max_iter_reached


In [17]:
jac_nochange = run_kmeans_forced(
    X, K, dist_fn=jaccard_dist,
    rule='no_change', max_iter=500, seed=7
)

print("JACCARD (no_change) -> SSE:", jac_nochange["sse"],
      "iters:", jac_nochange["it"], "time:", jac_nochange["time"],
      "stop:", jac_nochange["stop_reason"])


JACCARD (no_change) -> SSE: 3672.9324101615757 iters: 52 time: 111.71097710000322 stop: no_centroid_change


In [18]:
jac_sseinc = run_kmeans_forced(
    X, K, dist_fn=jaccard_dist,
    rule='sse_increase', max_iter=500, seed=7
)

print("JACCARD (sse_increase) -> SSE:", jac_sseinc["sse"],
      "iters:", jac_sseinc["it"], "time:", jac_sseinc["time"],
      "stop:", jac_sseinc["stop_reason"])


JACCARD (sse_increase) -> SSE: 3670.909111553253 iters: 21 time: 43.963881600000605 stop: sse_increase


In [19]:
jac_max100 = run_kmeans_forced(
    X, K, dist_fn=jaccard_dist,
    rule='max_iter', max_iter=100, seed=7
)

print("JACCARD (max_iter=100) -> SSE:", jac_max100["sse"],
      "iters:", jac_max100["it"], "time:", jac_max100["time"],
      "stop:", jac_max100["stop_reason"])


JACCARD (max_iter=100) -> SSE: 3672.9324101615757 iters: 100 time: 208.2765026000052 stop: max_iter_reached
